In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce

In [3]:
df = pd.read_csv(r"https://raw.githubusercontent.com/JonathanBechtel/dat-master/main/Data/Unit2/restaurant_data/master.csv?token=AADVTID37SY7Y7UBG24DT2DB2YIYG", parse_dates = ['visit_date'])

In [27]:
df.select_dtypes(include = np.object).columns.tolist()

<ipython-input-27-d8ff862afb65>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df.select_dtypes(include = np.object).columns.tolist()


['id', 'calendar_date', 'day_of_week', 'genre', 'area']

In [9]:
df['yesterday'] = df.groupby('id').apply(lambda x: x['visitors'].shift()).values
df['last_week'] = df.groupby('id').apply(lambda x: x['visitors'].shift(7)).values

# fill in missing reservations
df['reserve_visitors'] = df['reserve_visitors'].fillna(0)

# drop missing values from shifts
df = df.dropna()

In [10]:
X = df[['id', 'yesterday', 'last_week', 'reserve_visitors', 'day_of_week', 'holiday', 'area']]
y = df['visitors']

In [15]:
# let's assume these were our optimized parameters
tree = DecisionTreeRegressor(max_depth = 7, max_features = 0.8, min_samples_leaf = 10)

pipe = make_pipeline(ce.TargetEncoder(), tree)

In [16]:
# fit the tree, and export it
pipe.fit(X, y)

C:\Users\Jonat\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'area'])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=7, max_features=0.8,
                                       min_samples_leaf=10))])

In [19]:
# the pickle module allows you to export saved models
import pickle

# rb -- WRITE the file in BYTES
with open('pipe.pkl', 'wb') as export:
    # this creates an external version of the file that we can now import later on
    pickle.dump(pipe, export)

In [22]:
# we can now import this, and re-use it on new data:  very handy

with open('pipe.pkl', 'rb') as import_:
    # this will import the pickled object again
    pipe2 = pickle.load(import_)

In [23]:
# here it is
pipe2

Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'area'])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=7, max_features=0.8,
                                       min_samples_leaf=10))])

In [24]:
# and we can use it to make new predictions
pipe2.predict(X)

array([20.37353298, 24.71952428, 33.478     , ..., 57.2569198 ,
       41.83112497, 57.2569198 ])